In [17]:
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

In [18]:
data = pd.read_excel("/Users/ziyuewang/Desktop/Inference Analytics/complain data/User complain input_0202_top11.xlsx")

In [19]:
data["common complains"] = data["common complains"].map(lambda x: x.split("\n"))

In [20]:
data = data.explode("common complains")

In [21]:
#data.to_excel("User complain input_0202_top11_explode.xlsx")

In [22]:
X,y = data["common complains"],data["specialities"]

In [23]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [24]:
X_embedding = model.encode(X.values)

In [25]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_label = le.fit_transform(y)

In [26]:
y_label

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  4,  4,  4,  4,  4,  4,  4,  4,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  8,  8,  8,  8,  8,  8,  8,  8,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7])

In [34]:
dic = {0:"Cardiology",1:"Dermatology", 2:'Internal Medicine', 3: 'Neurology', 4: 'Obstetrics',
       5:"Obstetrics and Gynecology (OBGYN)",6:'Ophthalmology',7:'Optometry',8:'Pediatric Cardiology',
      9:'Pediatric Dermatology',10:'Primary Care'}

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
x_train, x_test, y_train, y_test = train_test_split(X_embedding, y_label, test_size=0.3, random_state=42)

In [37]:
svc = SVC()

In [38]:
clf = svc.fit(x_train,y_train)

In [39]:
y_pred = clf.predict(x_test)

In [40]:
from sklearn import metrics

In [41]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7083333333333334


In [42]:
from sklearn.metrics import multilabel_confusion_matrix as mcm, classification_report

In [46]:
print(classification_report(y_test, y_pred,target_names = ['Cardiology', 'Dermatology', 'Internal Medicine', 'Neurology',
       'Obstetrics', 'Obstetrics and Gynecology (OBGYN)', 'Ophthalmology',
       'Optometry', 'Pediatric Cardiology', 'Pediatric Dermatology',
       'Primary Care']))

                                   precision    recall  f1-score   support

                       Cardiology       0.82      1.00      0.90         9
                      Dermatology       0.57      1.00      0.73         4
                Internal Medicine       0.00      0.00      0.00         3
                        Neurology       0.90      1.00      0.95         9
                       Obstetrics       1.00      0.50      0.67         2
Obstetrics and Gynecology (OBGYN)       0.83      1.00      0.91         5
                    Ophthalmology       0.50      1.00      0.67         4
                        Optometry       0.00      0.00      0.00         3
             Pediatric Cardiology       0.00      0.00      0.00         4
            Pediatric Dermatology       0.00      0.00      0.00         3
                     Primary Care       0.40      1.00      0.57         2

                         accuracy                           0.71        48
                       

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Augment the train dataset

In [47]:
data = pd.read_excel("/Users/ziyuewang/Desktop/Inference Analytics/complain data/User complain input_0202_top11.xlsx")

In [48]:
data.head()

,common complains,specialities
0,Chest pain or discomfort (angina)\nShortness o...,Cardiology
1,Abnormal vaginal bleeding or discharge\nMenstr...,Obstetrics and Gynecology (OBGYN)
2,Pregnancy-related issues such as morning sickn...,Obstetrics
3,Atopic dermatitis or eczema\nAcne\nPsoriasis\n...,Pediatric Dermatology
4,"Acne, rosacea, and other skin conditions\nEcze...",Dermatology


In [49]:
data["common complains"] = data["common complains"].map(lambda x: x.split("\n"))

In [50]:
data= data.explode("common complains")

In [51]:
X,y = data["common complains"], data["specialities"]

In [52]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [53]:
expand_data = pd.read_excel("/Users/ziyuewang/Desktop/Inference Analytics/complain data/User complain input_0202_top11_expanded.xlsx")

In [54]:
expand_data.drop(columns = ["Unnamed: 0"],axis=1,inplace = True)

In [55]:
expand_data

,common complains,specialities
0,Chest aches visit_www.ncfta.org discomfort (re...,Cardiology
1,Atrial fibrillation,Cardiology
2,Cardiac Nathan_Helburn,Cardiology
3,Angevine_Middle blodd presssure (hypertension ),Cardiology
4,Heart attach,Cardiology
...,...,...
1312,"Eye strain, headaches, e fatigue whih reading ...",Optometry
1313,Eye injuries home@timesdispatch.com spokespers...,Optometry
1314,Dizzying changhings Yandarbiyev_Chechnya_actin...,Optometry
1315,Sudden change ein vision ar loss of vision,Optometry


In [56]:
train_data = expand_data[~expand_data['common complains'].isin(x_test)]

In [57]:
train_data.shape

(1236, 2)

In [58]:
x_train, y_train = train_data["common complains"], train_data["specialities"]

In [59]:
x_train_embedding = model.encode(x_train.values)

In [60]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train_label = le.fit_transform(y_train)

In [61]:
svc = SVC()

In [62]:
clf = svc.fit(x_train_embedding, y_train_label)

In [63]:
x_test_embedding = model.encode(x_test.values)

In [64]:
y_pred = clf.predict(x_test_embedding)

In [65]:
y_test_label = le.fit_transform(y_test)

In [66]:
from sklearn import metrics

In [67]:
print("Accuracy:",metrics.accuracy_score(y_test_label, y_pred))

Accuracy: 0.9375


In [68]:
from sklearn.metrics import multilabel_confusion_matrix as mcm, classification_report

In [70]:
print(classification_report(y_test_label, y_pred,target_names = ['Cardiology', 'Dermatology', 'Internal Medicine', 'Neurology',
       'Obstetrics', 'Obstetrics and Gynecology (OBGYN)', 'Ophthalmology',
       'Optometry', 'Pediatric Cardiology', 'Pediatric Dermatology',
       'Primary Care']))

                                   precision    recall  f1-score   support

                       Cardiology       1.00      1.00      1.00        14
                      Dermatology       0.86      0.86      0.86         7
                Internal Medicine       1.00      0.80      0.89         5
                        Neurology       1.00      1.00      1.00        14
                       Obstetrics       0.80      1.00      0.89         4
Obstetrics and Gynecology (OBGYN)       1.00      0.80      0.89        10
                    Ophthalmology       1.00      1.00      1.00        12
                        Optometry       1.00      1.00      1.00         4
             Pediatric Cardiology       1.00      1.00      1.00         4
            Pediatric Dermatology       0.67      0.67      0.67         3
                     Primary Care       0.60      1.00      0.75         3

                         accuracy                           0.94        80
                       